In [8]:
import numpy as np
import random
from sympy import symbols, diff, lambdify
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical   
from keras import layers, regularizers
from keras.layers import *
import matplotlib.pyplot as plot
from timeit import default_timer
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier, KerasRegressor

In [9]:
classes = 10
input_shape = (32, 32, 3)
n=5000
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[1:n]
y_train = y_train[1:n]
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = to_categorical(y_train, classes)
y_test = to_categorical(y_test, classes)

In [10]:
def nn_model(alpha, beta1, beta2, init='uniform'):
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(classes, activation="softmax"),
        ]
    )
    
    adam = tf.keras.optimizers.Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
    model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

    return model

In [12]:
def fit_nn(model):

  nmodel = KerasClassifier(build_fn=model, verbose=1)

  batch_size = [32, 64, 128, 512]
  epochs = [10, 20, 50, 100]

  params = dict(epochs=epochs, batch_size=batch_size)

  gsresult = GridSearchCV(estimator=nmodel, param_grid=params, cv=3)
  final_model = gsresult.fit(x_train, y_train)
  mts = final_model.cv_results_['mean_test_score']
  sts = final_model.cv_results_['std_test_score']
  params = final_model.cv_results_['params']
  for mean, st, param in zip(mts, sts, params):
      print(f'{mean:.4} || {st:.4} || {param}')

In [ ]:
def eval_nn(model, batch_size, epochs):
    modelh = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    
    # loss plot
    plot.plot(np.sqrt(modelh.history['loss']), 'red', label='Training data')
    plot.plot(np.sqrt(modelh.history['val_loss']), 'blue', label='Test data')
    plot.title('Model Loss over time')
    plot.xlabel('Iterations')
    plot.ylabel('Loss')
    plot.legend()
    plot.show()
  
    # Accuracy plot
    plot.plot(np.sqrt(modelh.history['accuracy']), 'black', label='Training data')
    plot.plot(np.sqrt(modelh.history['val_accuracy']), 'Purple', label='Test data')
    plot.title('Accuracy plot for SGD')
    plot.ylabel('Accuracy')
    plot.xlabel('Iterations')
    plot.legend()
    plot.show()
    
    return (np.sqrt(modelh.history['accuracy']), np.sqrt(modelh.history['val_accuracy']))

In [ ]:
model = nn_model(0.001, 0.9, 0.999)
fit_nn(model)

In [ ]:
alpha_vals = [0.001, 1]
beta1_vals = [0.5, 0.999]
beta2_vals = [0.8, 0.999]
batch_size = 32
epochs = 100

for alpha in alpha_vals:
  for beta1 in beta1_vals:
    for beta2 in beta2_vals:
        model = nn_model(alpha, beta1, beta1)
        train_a, testa = eval_nn(model, batch_size, epochs)

In [ ]:
def random_search(fn, x_range, N, n):
    start = default_timer()
    X = []
    cost_x = []
    func_vals = []
    mcf = float('inf')
    for i in range(N):
        for j in range(n):
            cost_x.insert(j, np.random.uniform(low=x_range[0], high=x_range[1]+1))
        cost_function = fn(cost_x)
        if (cost_function < mcf):
            X = np.append(X, cost_x, axis=0)
            func_vals = np.append(func_vals, cost_function)
            mcf = cost_function
    run_time = default_timer() - start
    return (X, func_vals, run_time)

In [ ]:
def sgd_nn(rate, momentum):
  # Define neural model
  model = keras.Sequential(
      [
          keras.Input(shape=(28, 28, 1)),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(classes, activation="softmax"),
      ]
  )


  # Create SGD optimizer with given hyperparameters
  sgd = tf.keras.optimizers.SGD(lr=rate, momentum=momentum, nesterov=False)
  model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

  return model

In [ ]:
def sgd_eval(model, batch_size, epochs):
  modelh = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    
  # loss plot
  plot.plot(np.sqrt(modelh.history['loss']), 'red', label='Train')
  plot.plot(np.sqrt(modelh.history['val_loss']), 'blue', label='Test')
  plot.title('Loss plot for SGD')
  plot.ylabel('Loss')
  plot.xlabel('Iterations')
  plot.legend()
  plot.show()

  # Accuracy plot
  plot.plot(np.sqrt(modelh.history['accuracy']), 'black', label='Training data')
  plot.plot(np.sqrt(modelh.history['val_accuracy']), 'Purple', label='Test data')
  plot.title('Accuracy plot for SGD')
  plot.ylabel('Accuracy')
  plot.xlabel('Iterations')
  plot.legend()
  plot.show()

  return (np.sqrt(modelh.history['accuracy']), np.sqrt(modelh.history['val_accuracy']))

In [ ]:
rate_vals = [0.0001, 1]
momentum_vals = [0.5, 0.8]
batch_size = 32
epochs = 100

for rate in rate_vals:
  for momentum in momentum_vals:
    model_tuning_SGD = sgd_nn(rate=rate, momentum=momentum)
    accuracy_train_SGD, accuracy_test_SGD = sgd_eval(model=model_tuning_SGD, batch_size = batch_size, epochs = epochs)
